In [1]:
import aug_util as aug
import wv_util as wv
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import csv
import tqdm
%matplotlib inline

In [6]:
lines = glob.glob("data_xview/train/images/*")
with open("data_xview/xview_img.txt", mode='w', encoding='utf-8') as myfile:
    myfile.write('\n'.join(lines))